In [ ]:
!pip install -q transformers evaluate accelerate datasets pyzotero
from huggingface_hub import notebook_login
from transformers import AutoTokenizer

notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
from pyzotero import zotero
from google.colab import userdata
import pandas as pd

library_id = 7428931
library_type = 'user'
api_key = userdata.get('Zotero')
label2id = {"1 - ML Protein Engineering": 1, "2 - Meiosis": 2, "3 - Chromatin": 3, "4 - Microscopy": 4, "5 - ML Code": 5, "6 - Genome Engineering": 6, "7 - Protein Engineering": 7}

zot = zotero.Zotero(library_id, library_type, api_key)
collections = zot.collections()

MyBiblio = {'title': [], 'abstract': [], 'label': []}
types_item = []

for folder in collections:
  label = 0
  if folder['data']['name'] in list(label2id.keys()):
    label = label2id[folder['data']['name']]
  entries = zot.everything(zot.collection_items(folder['key'])) #the zot.everything wrapper allows to return all the elements and not only the first 100 as a regular pyzotero call would
  for reference in entries:
    if reference['data']['itemType'] != 'attachment':
      types_item.append(reference['data']['itemType'])
    if reference['data']['itemType'] in ['preprint','conferencePaper','journalArticle','report']:
      MyBiblio['title'].append(reference['data']['title'])
      MyBiblio['abstract'].append(reference['data']['abstractNote'])
      MyBiblio['label'].append(label)
#print(set(types_item))

df = pd.DataFrame.from_dict(MyBiblio)
df = df.dropna(subset="label").dropna(subset="abstract")
print(df)

                                                  title  \
0      Engineering soluble T-cell receptors for therapy   
1     Hyperstable Synthetic Mini-Proteins as Effecti...   
2     Stable, soluble T-cell receptor molecules for ...   
3     Structure-Based, Rational Design of T Cell Rec...   
4     Rational design of a CD4 mimic that inhibits H...   
...                                                 ...   
2918  Easy quantitative assessment of genome editing...   
2919  A transcription activator-like effector toolbo...   
2920   Genome engineering using the CRISPR-Cas9 system.   
2921  An inhibitor of nonhomologous end-joining abro...   
2922  Improving CRISPR-Cas nuclease specificity usin...   

                                               abstract  label  
0     Immunotherapy approaches that target peptide–h...      7  
1     Small, single-domain protein scaffolds are com...      7  
2                                                            7  
3     Adoptive cell transfer us

In [ ]:
from datasets import Dataset

df_null = pd.read_csv("/content/drive/MyDrive/WIP/Biblio/biblio.tsv", sep='\t', encoding="ISO-8859-1").dropna(subset="label").dropna(subset="abstract")
#this file contains a previous version of my biblio including some papers that had been validated to not be relevant to populate the non relevant category
df_null['label'] = df_null['label'].astype(int)
df_null = df_null[df_null['label'] == 0]
df_null_dataset = df_null[["abstract","label"]]
df_dataset = df[["abstract","label"]]
df_total = pd.concat([df_null_dataset, df_dataset])
dataset = Dataset.from_pandas(df_total)
#dataset = dataset.remove_columns(["__index_level_0__"])

dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

def preprocess_function(examples):
    return tokenizer(examples["abstract"], truncation=True, max_length=512)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext")
tokenized_bilio = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Map:   0%|          | 0/2338 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "Not Relevant", 1: "ML Protein Engineering", 2: "Meiosis", 3: "Chromatin", 4: "Microscopy", 5: "ML Code", 6: "Genome Engineering", 7: "Protein Engineering"}
label2id = {"Not Relevant": 0, "ML Protein Engineering": 1, "Meiosis": 2, "Chromatin": 3, "Microscopy": 4, "ML Code": 5, "Genome Engineering": 6, "Protein Engineering": 7}

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext", num_labels=8, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="PubMed_interests2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bilio["train"],
    eval_dataset=tokenized_bilio["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.525552,0.839316
2,No log,0.429867,0.861538
3,No log,0.436930,0.858120
4,0.525100,0.445809,0.864957


TrainOutput(global_step=588, training_loss=0.4895321592992666, metrics={'train_runtime': 741.3551, 'train_samples_per_second': 12.615, 'train_steps_per_second': 0.793, 'total_flos': 1778053321311840.0, 'train_loss': 0.4895321592992666, 'epoch': 4.0})